In [1]:
#import library stuff
import warnings

from numpy.lib.function_base import average
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import math
import logging
logging.basicConfig(level=logging.ERROR)

import matplotlib.pyplot as plt

import os 

In [2]:
#Network parameters
n_input = 12
n_hidden1 = 24
n_hidden2 = 12
n_hidden3 = 6
n_output = 2

#Learning parameters
learning_constant = 0.2 
number_epochs = 20000
batch_size = 1000

#Defining the input and the output
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])

#DEFINING WEIGHTS AND BIASES
b1 = tf.Variable(tf.random_normal([n_hidden1]))
b2 = tf.Variable(tf.random_normal([n_hidden2]))
b3 = tf.Variable(tf.random_normal([n_hidden3]))
b4 = tf.Variable(tf.random_normal([n_output]))
w1 = tf.Variable(tf.random_normal([n_input, n_hidden1]))
w2 = tf.Variable(tf.random_normal([n_hidden1, n_hidden2]))
w3 = tf.Variable(tf.random_normal([n_hidden2, n_hidden3]))
w4 = tf.Variable(tf.random_normal([n_hidden3, n_output]))

def multilayer_perceptron(input_d):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(input_d, w1), b1)) #f(X * W1 + b1)
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, w2), b2))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, w3), b3))
    out_layer = tf.nn.softmax(tf.add(tf.matmul(layer_3, w4),b4))
    return out_layer


In [3]:
def loadHeartFailureDataset():    #just get the data

    path = os.path.join("datasets","heart_failure_clinical_records_dataset.csv")
    data = np.genfromtxt(path, delimiter=",", names=True)

    #splitting data and label out
    data = data.view(np.float64).reshape((len(data), -1))
    x_data = data[:, 0:-1]
    y_data = data[:, -1]
    y_data = y_data.astype('int32')
    y_data = np.identity(2)[y_data] # one hot encoding

    # print(x_data)
    # print(y_data)
    return x_data, y_data

def minMaxNorm(data): #normalize the data
    spacing = 5
    pct_min = np.percentile(data, spacing, axis=0)
    pct_max = np.percentile(data, 100 - spacing, axis=0)
    norm_data = np.zeros(data.shape)
    for i in range(len(data[0])):
        norm_data[:, i] = (data[:,i] - pct_min[i]) / (pct_max[i] - pct_min[i])

    norm_data[norm_data > 1] = 1
    norm_data[norm_data < 0] = 0

    return norm_data

def splitLabel(data, number_of_labels):
    number_of_columns = len(data[0])
    
    x_data = data[:, 0:-number_of_labels]
    y_data = data[:, number_of_columns - number_of_labels : number_of_columns]
    return x_data, y_data

def mergeLabel(x_data, y_data):
    return np.column_stack((x_data, y_data))

def validationSplit(data, percentage): #split the data between train and test
    portion = round(len(data)*percentage)
    train = data[0:portion]
    test = data[portion:len(data)]
    return train, test

def partitionIndex(instances_size, k): 
    partitionIndicies = np.zeros((k, 2))
    itemsPerFold = round(instances_size / k)
    for i in range(k): 
        if i == 0:
            partitionIndicies[i, :] = [(i)*itemsPerFold, (i + 1)*itemsPerFold - 1]
        else:
            partitionIndicies[i, :] = [(i)*itemsPerFold, (i + 1)*itemsPerFold - 1]
        
    partitionIndicies[k - 1,1] = instances_size - 1
    return partitionIndicies

def splitPartition(data, range_indices, partition_selection):
    range_list = np.arange(range_indices[partition_selection,0],range_indices[partition_selection,1], dtype = int)
    is_test_mat = np.zeros((data.shape[0], 1), dtype = int)
    
    for i in range_list:
        is_test_mat[i] = 1
        
    test_data = np.empty((0,data.shape[1]))
    train_data = np.empty((0,data.shape[1]))

    for i in range(data.shape[0]):
        if(is_test_mat[i] == 1):
            test_data = np.append(test_data, [data[i,:]], axis = 0)
        else:
            train_data = np.append(train_data, [data[i,:]], axis = 0)
       
    return test_data, train_data
    
    
    

In [4]:
#Create model
neural_network = multilayer_perceptron(X)

#Define loss and optimizer
loss_op = tf.reduce_mean(tf.math.squared_difference(neural_network,Y))
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=neural_network,labels=Y))


#Initializing the variables
init = tf.global_variables_initializer()

# Load dataset
x_data, y_data = loadHeartFailureDataset()
x_data = minMaxNorm(x_data)

number_of_labels = len(y_data[0]) #because of the output required 2
# train_x = x_data
# train_y = y_data

data = mergeLabel(x_data, y_data)
#train_set, test_set = validationSplit(data, 0.5)
#train_x, train_y = splitLabel(train_set, number_of_labels)
#test_x, test_y = splitLabel(test_set, number_of_labels)
# print("\nTrain X:\n", train_x)
# print("\nTrain Y:\n", train_y)
# print("\nTest X:\n", test_x)
# print("\nTest Y:\n", test_y)

with tf.Session() as sess:
    sess.run(init)

    k = 10
    inner_k = 5
    
    accuracies = np.zeros((1, k))
    f1scores = np.zeros((1, k))
    all_best_hyper_learning = np.zeros((k, inner_k))
    all_best_hyper_epochs = np.zeros((k,inner_k))
    
    #setup outer partition
    outer_range_indices = partitionIndex(data.shape[0], k)
  
    for i in range(k): 
        #create outer partition
        outer_test_data, outer_train_data = splitPartition(data, outer_range_indices, i)
        
        #setup inner partition
        inner_range_indices = partitionIndex(outer_train_data.shape[0], inner_k)
        
        for j in range(inner_k):
            #create inner partition
            inner_test_data, inner_train_data = splitPartition(outer_train_data, inner_range_indices, j)
            
            
            inner_train_X, inner_train_Y = splitLabel(inner_train_data, number_of_labels)
            inner_test_X, inner_test_Y = splitLabel(inner_test_data, number_of_labels)
            
            #grid search
            for hyper_learning in np.arange(0.1,1.1,0.1):
                for hyper_epoch in range(10000, 110000, 10000):
                    
                    optimizer = tf.train.GradientDescentOptimizer(hyper_learning).minimize(loss_op)
                    for epoch in range(hyper_epoch):
                        sess.run(optimizer, feed_dict={X: inner_train_X, Y: inner_train_Y})
                        
                        #Display the epoch
                        if epoch % 100 == 0:
                            print("Epoch:", '%d' % (epoch))
                    
                    pred = (neural_network)
                    output = neural_network.eval({X: inner_test_X})
                    print(output)
                    break
                    correct_prediction1 = tf.equal(tf.argmax(output, 1),inner_train_Y)
                    accuracy1 = tf.reduce_mean(tf.cast(correct_prediction1, tf.float32))
                    print(accuracy1)
                    #print(accuracy1.eval({X: batch_x}))
                    
                    
                     
        
       
          
   
    
    

Epoch: 0
Epoch: 100
Epoch: 200
Epoch: 300
Epoch: 400
Epoch: 500
Epoch: 600
Epoch: 700
Epoch: 800
Epoch: 900
Epoch: 1000
Epoch: 1100
Epoch: 1200
Epoch: 1300
Epoch: 1400
Epoch: 1500
Epoch: 1600
Epoch: 1700
Epoch: 1800
Epoch: 1900
Epoch: 2000
Epoch: 2100
Epoch: 2200
Epoch: 2300
Epoch: 2400
Epoch: 2500
Epoch: 2600
Epoch: 2700
Epoch: 2800
Epoch: 2900
Epoch: 3000
Epoch: 3100
Epoch: 3200
Epoch: 3300
Epoch: 3400
Epoch: 3500
Epoch: 3600
Epoch: 3700
Epoch: 3800
Epoch: 3900
Epoch: 4000
Epoch: 4100
Epoch: 4200
Epoch: 4300
Epoch: 4400
Epoch: 4500
Epoch: 4600
Epoch: 4700
Epoch: 4800
Epoch: 4900
Epoch: 5000
Epoch: 5100
Epoch: 5200
Epoch: 5300
Epoch: 5400
Epoch: 5500
Epoch: 5600
Epoch: 5700
Epoch: 5800
Epoch: 5900
Epoch: 6000
Epoch: 6100
Epoch: 6200
Epoch: 6300
Epoch: 6400
Epoch: 6500
Epoch: 6600
Epoch: 6700
Epoch: 6800
Epoch: 6900
Epoch: 7000
Epoch: 7100
Epoch: 7200
Epoch: 7300
Epoch: 7400
Epoch: 7500
Epoch: 7600
Epoch: 7700
Epoch: 7800
Epoch: 7900
Epoch: 8000
Epoch: 8100
Epoch: 8200
Epoch: 8300
Epoc